In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os 
import numpy as np
import pandas as pd 

panda_data = pd.read_csv('/content/gdrive/MyDrive/FYP/YOLOv5/YOLOv5-640/sorted_combined_result.csv')  

In [4]:
panda_data

,Unnamed: 0,xmin,ymin,xmax,ymax,confidence,class,name,num
0,7,208.091995,306.099640,280.373108,419.799835,0.632203,1,steps,6
1,13,456.328033,342.540497,490.871429,369.541840,0.397965,1,steps,12
2,12,472.819702,339.634155,495.667603,371.752747,0.565076,1,steps,12
3,11,450.234924,344.158020,467.563049,369.608459,0.596376,1,steps,12
4,17,123.958984,333.077393,148.525391,384.484009,0.702842,1,steps,16
...,...,...,...,...,...,...,...,...,...
427,328,125.217079,341.969421,146.243317,363.651001,0.488919,1,steps,1283
428,348,139.365631,338.552856,168.460907,363.300415,0.386606,1,steps,1292
429,402,459.813843,329.244232,508.830811,459.008148,0.305557,1,steps,1324
430,414,65.962379,345.122620,93.991005,370.488098,0.656088,1,steps,1332


In [5]:
# calculate the area of bounding box
panda_data['area'] = (panda_data['xmax'] - panda_data['xmin']) * (panda_data['ymax'] - panda_data['ymin'])
# calculate the postion of the meddle of bounding box
panda_data['position'] = (panda_data['xmax'] + panda_data['xmin']) / 2

# abandon information not use
panda_data.pop('xmin')
panda_data.pop('xmax')
panda_data.pop('ymin')
panda_data.pop('ymax')
panda_data.pop('Unnamed: 0')
#panda_data.pop('confidence')

0        7
1       13
2       12
3       11
4       17
      ... 
427    328
428    348
429    402
430    414
431    418
Name: Unnamed: 0, Length: 432, dtype: int64

In [6]:
panda_data

,confidence,class,name,num,area,position
0,0.632203,1,steps,6,8218.376628,244.232552
1,0.397965,1,steps,12,932.718076,473.599731
2,0.565076,1,steps,12,733.842375,484.243652
3,0.596376,1,steps,12,441.008396,458.898987
4,0.702842,1,steps,16,1262.875818,136.242188
...,...,...,...,...,...,...
427,0.488919,1,steps,1283,455.882042,135.730198
428,0.386606,1,steps,1292,720.037045,153.913269
429,0.305557,1,steps,1324,6360.633689,484.322327
430,0.656088,1,steps,1332,710.959498,79.976692


In [ ]:
# read metadata from images

!pip3 install Pillow
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

# path to the image or video
img_path = "/content/gdrive/MyDrive/FYP/ICL_Panorama/SV-1-51.495397-0.178344-97.jpg"

# read the image data using PIL
image = Image.open(img_path)

# extract other basic metadata
info_dict = {
    "Filename": image.filename,
    "Image Size": image.size,
    "Image Height": image.height,
    "Image Width": image.width,
    "Image Format": image.format,
    "Image Mode": image.mode,
    "Image is Animated": getattr(image, "is_animated", False),
    "Frames in Image": getattr(image, "n_frames", 1)
}

for label,value in info_dict.items():
    print(f"{label:25}: {value}")

# extract EXIF data
exifdata = image.getexif()

# iterating over all EXIF data fields
for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
    tag = TAGS.get(tag_id, tag_id)
    data = exifdata.get(tag_id)
    # decode bytes 
    if isinstance(data, bytes):
        data = data.decode()
    print(f"{tag:25}: {data}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Filename                 : /content/gdrive/MyDrive/FYP/ICL_Panorama/SV-1-51.495397-0.178344-97.jpg
Image Size               : (1024, 512)
Image Height             : 512
Image Width              : 1024
Image Format             : JPEG
Image Mode               : RGB
Image is Animated        : False
Frames in Image          : 1
GPSInfo                  : {1: 'N', 2: ((51, 1), (29, 1), (4343, 100)), 3: 'W', 4: ((0, 1), (10, 1), (4204, 100)), 5: b'\x00', 6: ((9, 1), (4, 1), (3475, 100)), 17: ((83, 1), (29, 1), (1128, 100))}
UserComment              : Downloaded with Street View Download 360. Visit svd360.com for more info. Panorama ID: Ka-A22xqTE2yQojmuIMbfQ
ExifOffset               : 192
ProcessingSoftware       : SVD360 4.0.12
ImageID                  : Ka-A22xqTE2yQojmuIMbfQ
Make                     : RICOH
Model                    : RICOH THETA S
ImageNumber              : 1
Software       

In [ ]:
class Metadata(object):
    def __init__(self, img):
        self.img = img
        self.exif_data = self.get_exif_data()
        self.lat = self.get_lat()
        self.lon = self.get_lon()
        self.nor_ang = self.get_rot()
        self.id = self.get_id()
        super(Metadata, self).__init__()

    @staticmethod
    def get_if_exist(data, key):
        if key in data:
            return data[key]
        return None

    @staticmethod
    def convert_to_degress(value):
        """Helper function to convert the GPS coordinates
        stored in the EXIF to degress in float format"""
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)
        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)

        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)

        return d + (m / 60.0) + (s / 3600.0)

    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also
        converts the GPS Tags"""
        exif_data = {}
        info = self.img._getexif()
        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        return exif_data

    def get_lat(self):
        """Returns the latitude and longitude, if available, from the 
        provided exif_data (obtained through get_exif_data above)"""
        if 'GPSInfo' in self.exif_data:
            gps_info = self.exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            if gps_latitude and gps_latitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":
                    lat = 0 - lat
                lat = str(f"{lat:.{6}f}")
                return lat
        else:
            return None

    def get_lon(self):
        """Returns the latitude and longitude, if available, from the 
        provided exif_data (obtained through get_exif_data above)"""
        if 'GPSInfo' in self.exif_data:
            gps_info = self.exif_data["GPSInfo"]
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_longitude and gps_longitude_ref:
                lon = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lon = 0 - lon
                lon = str(f"{lon:.{6}f}")
                return lon
        else:
            return None

    def get_rot(self):
      """Returns the north rotation angle, if available, from the 
      provided exif_data (obtained through get_exif_data above)"""
      if 'GPSInfo' in self.exif_data:
          gps_info = self.exif_data["GPSInfo"]
          gps_northang = self.get_if_exist(gps_info, 'GPSImgDirection')
          if gps_northang:
              nor_ang = self.convert_to_degress(gps_northang)
              if nor_ang > 180:
                nor_ang = str(f"{(180 - nor_ang + 360):.{6}f}")
              else:
                nor_ang = str(f"{(180 - nor_ang):.{6}f}")
              return nor_ang
      else:
          return None

    def get_id(self):
        if 'ImageID' in self.exif_data:
            id = self.exif_data['ImageID']
            return id 

In [ ]:
import glob

if __name__ == '__main__':
        imgs = [Image.open(file) for file in glob.glob("/content/gdrive/MyDrive/FYP/ICL_Panorama/*.jpg")]
        lat_set = []
        lon_set = []
        nor_ang_set = []
        id_set = []

        for i in panda_data['num']:
            img_metadata = Metadata(imgs[i-1])
            lat = img_metadata.lat
            lon = img_metadata.lon
            nor_ang = img_metadata.nor_ang
            id = img_metadata.id
            lat_set.append(lat)
            lon_set.append(lon)
            nor_ang_set.append(nor_ang)
            id_set.append(id)

# convert data type from string to float in Python list
# lat_set = [float(x) for x in lat_set]
# lon_set = [float(y) for y in lon_set]
# nor_ang_set = [float(z) for z in nor_ang_set]

print(lat_set)
print(lon_set)
print(nor_ang_set)
print(id_set)

['51.497356', '51.497594', '51.497594', '51.497594', '51.497675', '51.498103', '51.497919', '51.498072', '51.497950', '51.497950', '51.498342', '51.498253', '51.498403', '51.498281', '51.498233', '51.498389', '51.498364', '51.498336', '51.498378', '51.498456', '51.498475', '51.498356', '51.498356', '51.498356', '51.498478', '51.498478', '51.498497', '51.498306', '51.498306', '51.498306', '51.498500', '51.498500', '51.498489', '51.498508', '51.498508', '51.498494', '51.498494', '51.498492', '51.498492', '51.498492', '51.498506', '51.498506', '51.497900', '51.497900', '51.497922', '51.498200', '51.498228', '51.498228', '51.498222', '51.498206', '51.498214', '51.498386', '51.498439', '51.498475', '51.498475', '51.498658', '51.498567', '51.498186', '51.498186', '51.498175', '51.499667', '51.499544', '51.499369', '51.498983', '51.498983', '51.499072', '51.499161', '51.499161', '51.499250', '51.499250', '51.498661', '51.499106', '51.498917', '51.499050', '51.499064', '51.498953', '51.499053'

In [ ]:
import numpy as np

# convert degree to radius, convert string to float
lat_set = [(float(x)) for x in lat_set]
lon_set = [(float(y)) for y in lon_set]
nor_ang_set = [float(z) for z in nor_ang_set]

In [ ]:
panda_data['ID'] = id_set
panda_data['Lat'] = lat_set
panda_data['Lon'] = lon_set
panda_data['Rot'] = nor_ang_set

In [ ]:
panda_data

,confidence,class,name,num,area,position,ID,Lat,Lon,Rot
0,0.632203,1,steps,6,8218.376628,244.232552,J53ie2H_Zqpz_rMFrKET-g,51.497356,-0.177036,282.593750
1,0.397965,1,steps,12,932.718076,473.599731,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472
2,0.565076,1,steps,12,733.842375,484.243652,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472
3,0.596376,1,steps,12,441.008396,458.898987,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472
4,0.702842,1,steps,16,1262.875818,136.242188,WOcyal2c14l8yZrRFKtm-g,51.497675,-0.173950,8.852967
...,...,...,...,...,...,...,...,...,...,...
427,0.488919,1,steps,1283,455.882042,135.730198,M4nO0BTARCihsiuImToMxA,51.496889,-0.179400,187.424439
428,0.386606,1,steps,1292,720.037045,153.913269,GpSH5SgA82NUlSQMJZF3Ew,51.496794,-0.179383,187.153808
429,0.305557,1,steps,1324,6360.633689,484.322327,hLa_QdS8b4d5Yo7hI_Bkrw,51.497225,-0.175672,257.997192
430,0.656088,1,steps,1332,710.959498,79.976692,zOEpB2uI85swZMmug3fcrA,51.497897,-0.180019,276.646972


In [ ]:
# Simulation for the distance from the shooting point to the step/ramp 
# Assume the farest distance in the image of the step/ramp is 15 m
# The relation between the area of the bounding box (x) with the distance (y) is y = -15/(640*640)*x+15
reference_distance = 10
occupied_ratio = 0.2

panda_data['Distance'] = -reference_distance / (640*640*occupied_ratio) * panda_data['area'] + reference_distance

# Simulation for the direction of the step/ramp related to North
# The middle position of the bounding box is related to the image size (640x640), which has to be convert to degree(360)
panda_data['position(deg)'] = panda_data['position'] / 640 * 360
panda_data['Direction'] = panda_data['position(deg)'] - panda_data['Rot']

# convert degree to radians
panda_data['Direction'] = panda_data['Direction'] * 2 * np.pi / 360


In [ ]:
panda_data

,confidence,class,name,num,area,position,ID,Lat,Lon,Rot,Distance,position(deg),Direction
0,0.632203,1,steps,6,8218.376628,244.232552,J53ie2H_Zqpz_rMFrKET-g,51.497356,-0.177036,282.593750,8.996780,137.380810,-2.534444
1,0.397965,1,steps,12,932.718076,473.599731,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472,9.886143,266.399849,1.383564
2,0.565076,1,steps,12,733.842375,484.243652,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472,9.910420,272.387054,1.488060
3,0.596376,1,steps,12,441.008396,458.898987,95zUT9yOqbHuOVZx5QLc8A,51.497594,-0.173922,187.127472,9.946166,258.130680,1.239240
4,0.702842,1,steps,16,1262.875818,136.242188,WOcyal2c14l8yZrRFKtm-g,51.497675,-0.173950,8.852967,9.845840,76.636230,1.183041
...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0.488919,1,steps,1283,455.882042,135.730198,M4nO0BTARCihsiuImToMxA,51.496889,-0.179400,187.424439,9.944350,76.348236,-1.938645
428,0.386606,1,steps,1292,720.037045,153.913269,GpSH5SgA82NUlSQMJZF3Ew,51.496794,-0.179383,187.153808,9.912105,86.576214,-1.755410
429,0.305557,1,steps,1324,6360.633689,484.322327,hLa_QdS8b4d5Yo7hI_Bkrw,51.497225,-0.175672,257.997192,9.223555,272.431309,0.251923
430,0.656088,1,steps,1332,710.959498,79.976692,zOEpB2uI85swZMmug3fcrA,51.497897,-0.180019,276.646972,9.913213,44.986889,-4.043231


In [ ]:
# !!!!!!!!!
# !!!!!!!!!
# Original model (less accuracy)

# lon2 = lon1 + d*sinα / [ARC*cos(lat1)]
# lat2 = lat1 + d*cosα / (ARC)

# Equatorial radius
EARTH_RADIUS = 6371 *1000
# Length of the Earth's arc per degree
EARTH_ARC = EARTH_RADIUS * np.pi / 180

panda_data['Lat'] = panda_data['Lat'] + panda_data['Distance'] * np.cos(panda_data['Direction']) / (EARTH_ARC)
panda_data['Lon'] = panda_data['Lon'] + panda_data['Distance'] * np.sin(panda_data['Direction']) / (EARTH_ARC * np.cos(panda_data['Lat'] * np.pi / 180))

# convert latitude and longitude back to degree
# panda_data['Lat_deg'] = panda_data['Lat'] * 360 / 2 / np.pi
# panda_data['Lon_deg'] = panda_data['Lon'] * 360 / 2 / np.pi

In [ ]:
import math

R = 6371 *1000 #Radius of the Earth in meters
delta = panda_data['Distance']/R #angular distance d/R

panda_data['Lat'] = np.deg2rad(panda_data['Lat'])
panda_data['Lon'] = np.deg2rad(panda_data['Lon'])

lat2 = np.arcsin(np.sin(panda_data['Lat']) * np.cos(delta) + np.cos(panda_data['Lat']) * np.sin(delta) * np.cos(panda_data['Direction']))

lon2 = panda_data['Lon'] + np.arctan2(np.sin(panda_data['Direction']) * np.sin(delta) * np.cos(panda_data['Lat']),
             np.cos(delta) - np.sin(panda_data['Lat']) * np.sin(lat2))

panda_data['Lat'] = np.rad2deg(lat2)
panda_data['Lon'] = np.rad2deg(lon2)

In [ ]:
# abandon information not use

cols = ['ID', 'name', 'Lat', 'Lon', 'confidence']
panda_data_subset = panda_data[cols]

result_with_obj_Lat_Lon = panda_data_subset
result_with_obj_Lat_Lon.to_csv('/content/gdrive/MyDrive/FYP/YOLOv5/YOLOv5-640/result_with_obj_Lat_Lon.csv')

In [ ]:
panda_data_subset

,ID,name,Lat,Lon,confidence
0,J53ie2H_Zqpz_rMFrKET-g,steps,51.497290,-0.177110,0.632203
1,95zUT9yOqbHuOVZx5QLc8A,steps,51.497611,-0.173782,0.397965
2,95zUT9yOqbHuOVZx5QLc8A,steps,51.497601,-0.173779,0.565076
3,95zUT9yOqbHuOVZx5QLc8A,steps,51.497623,-0.173786,0.596376
4,WOcyal2c14l8yZrRFKtm-g,steps,51.497708,-0.173818,0.702842
...,...,...,...,...,...
427,M4nO0BTARCihsiuImToMxA,steps,51.496857,-0.179534,0.488919
428,GpSH5SgA82NUlSQMJZF3Ew,steps,51.496778,-0.179524,0.386606
429,hLa_QdS8b4d5Yo7hI_Bkrw,steps,51.497305,-0.175639,0.305557
430,zOEpB2uI85swZMmug3fcrA,steps,51.497842,-0.179907,0.656088


In [ ]:
# import json

# result_with_obj_Lat_Lon.to_json(path_or_buf = '/content/gdrive/MyDrive/FYP/YOLOv5/result_json.json', orient="index")
# parsed_json = json.loads(result_json)
# json.dumps(parsed_json, indent=4) 

In [ ]:
def df_to_geojson(df, properties, lat='Lat', lon='Lon'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [ ]:
cols = ['ID', 'name', 'Lat', 'Lon']
result_geojson = df_to_geojson(panda_data_subset, cols)

In [ ]:
!pip install json
import json

with open('/content/gdrive/MyDrive/FYP/YOLOv5/YOLOv5-640/result_geojson_0.3_0.2x10.geojson', 'w', encoding='utf-8') as f:
    json.dump(result_geojson, f, ensure_ascii=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
